# images 转 tfrecord
使用此脚本可以将图片转化为tfrecord格式的文件。

tensorflow >2.0.0

需要两个文件，一个是图片文件的路径，一个是标签例如：images.txt  labels.txt

主要用到两个命令读取和反编码图片。

img_raw = tf.io.read_file(image_path)

img_tensor = tf.image.decode_image(img_raw)

In [49]:
import tensorflow as tf


images_path_file = open('./images.txt', 'r')
images_path_list = []

labels = open('./labels.txt', 'r')

'''
判断文件路径是否含有换行符
'''
for i in images_path_file:
    if i[-1] == '\n':
        images_path_list.append(i[:-1])
    else:
        images_path_list.append(i)
        
labels_list = [int(x[0:-1]) for x in labels]
data = zip(images_path_list, labels_list)


In [14]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def image_example(image_string, label):
    image_shape = tf.image.decode_jpeg(image_string).shape
    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [15]:
record_file = 'tset.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for filename, label in data:
        image_string = open(filename, 'rb').read()
        tf_example = image_example(image_string, label)
        writer.write(tf_example.SerializeToString())

# 读取文件

下面的代码用于恢复图像信息，如果需要可以将下面的代码复制到你的代码中

In [36]:
import numpy as np


raw_image_dataset = tf.data.TFRecordDataset('tset.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    # 'height': tf.io.FixedLenFeature([], tf.int64),
    # 'width': tf.io.FixedLenFeature([], tf.int64),
    # 'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string)
}


def _parse_image_function(example_proto):
    '''
    恢复到数组形式，并将数组添加一个维度形成(None，height, weight, channels)的形式，方便下一步的训练
    '''
    image_raw = tf.io.parse_single_example(example_proto,
                                      {'image_raw': tf.io.FixedLenFeature([], tf.string)})['image_raw']
    image = tf.image.decode_jpeg(image_raw, channels=3)
    image = tf.image.resize(image, [227, 227])
    image = tf.expand_dims(image, axis=0)
    image /= 255.0  # normalize to [0,1] range

    return image


def _labels(example_proto):
    return tf.io.parse_single_example(example_proto,
                                      {'label': tf.io.FixedLenFeature([], tf.int64)})['label']

In [39]:
images = raw_image_dataset.map(_parse_image_function)
labels =  raw_image_dataset.map(_labels)
dataset = tf.data.Dataset.zip((images, labels))

## 导入模型中

Example

In [40]:
227# 建立模型
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(227, 227, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


# 进行编译和训练
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(dataset, epochs=2)

Epoch 1/2
79/79 [==============================] - 14s 183ms/step - loss: 0.0281 - accuracy: 1.0000
Epoch 2/2
79/79 [==============================] - 13s 170ms/step - loss: 0.0000e+00 - accuracy: 1.0000
